# Stable surface mask from hillshade of reference DEM

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xrspatial as xrs
import scipy
import os
import skimage
from shapely.geometry import LineString, Polygon, MultiPolygon
import geopandas as gpd

## Load DEM from file

In [ ]:
# Load DEM from file
data_path = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS'
# data_path = f'/Users/rdcrlrka/Research/PhD/SkySat-Stereo/study-sites/{site_name}/'
dem_fn = os.path.join(data_path, 'refdem', 'MCS_REFDEM_WGS84.tif')
dem_da = xr.open_dataarray(dem_fn).squeeze()
dem_ds = xr.open_dataset(dem_fn).squeeze()
dem_ds = dem_ds.rename({'band_data':'elevation'})
dem_ds

## Calculate slope and hillshade

In [ ]:
# Calculate slope and hillshade
dem_ds['slope'] = xrs.slope(dem_da)
dem_ds['hillshade'] = xrs.hillshade(dem_da, angle_altitude=25)
dem_ds

In [ ]:
# Plot
fig, ax = plt.subplots(1, 2, figsize=(12,6))
im = ax[0].imshow(dem_ds['slope'].data, cmap='PuOr', clim=(0,20),
                  extent=(np.min(dem_ds.x.data), np.max(dem_ds.x.data), np.min(dem_ds.y.data), np.max(dem_ds.y.data)))
fig.colorbar(im, ax=ax[0], label='degrees', shrink=0.8, orientation='horizontal')
ax[0].set_title('Slope')
im = ax[1].imshow(dem_ds['hillshade'].data, cmap='Greys', clim=(0,1),
                  extent=(np.min(dem_ds.x.data), np.max(dem_ds.x.data), np.min(dem_ds.y.data), np.max(dem_ds.y.data)))
fig.colorbar(im, ax=ax[1], label='reflectance', shrink=0.8, orientation='horizontal')
ax[1].set_title('Hillshade')                  

plt.show()

## Test and apply threshold(s) for elevation and slope

In [ ]:
# Select threshold values
thresh_elev = 1900
thresh_slope = 5

# Apply threshold values
dem_thresh = xr.where((dem_ds['elevation'] < thresh_elev) & (dem_ds['slope'] < thresh_slope), 1, 0)

# Fill holes and smooth the thresholded image
dem_thresh_filled = scipy.ndimage.binary_fill_holes(dem_thresh.data)
dem_thresh_filled_smoothed = skimage.filters.gaussian(dem_thresh_filled, sigma=5)

# Plot
plt.figure(figsize=(12,12))
plt.imshow(dem_thresh_filled_smoothed, cmap='Greys')
plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# Select threshold values
# thresh_elev = 1900
# thresh_slope = 5

# # Apply threshold values
# dem_thresh = xr.where((dem_ds['elevation'] < thresh_elev) & (dem_ds['slope'] < thresh_slope), 1, 0)

# # Fill holes and smooth the thresholded image
# dem_thresh_filled = scipy.ndimage.binary_fill_holes(dem_thresh.data)
# dem_thresh_filled_smoothed = skimage.filters.gaussian(dem_thresh_filled, sigma=5)

# # Identify contours in thresholded image
# contours = skimage.measure.find_contours(dem_thresh_filled)
# print(f'{len(contours)} contours found')

# Convert contour coordinates to map coordinates and create polygons
x_res = dem_ds.x.data[1] - dem_ds.x.data[0]
y_res = dem_ds.y.data[1] - dem_ds['y'][0]).item()
x_min = dem_ds['x'].min().item()
y_max = dem_ds['y'].max().item()  # y_max because y coordinates are typically decreasing

# Define a function to convert image coordinates to map coordinates
def image_to_map_coordinates(x, y, x_res, y_res, x_min, y_max):
    map_x = x_min + x * x_res
    map_y = y_max - y * y_res  # minus because image coordinates start from top left
    return map_x, map_y

polys_list = []
for contour in contours:
    contour_map_coords = np.array([image_to_map_coordinates(x, y, x_res, y_res, x_min, y_max) for x, y in contour])
    polys_list.append(Polygon(contour_map_coords))


## Identify contours in the thresholded image, filter, and polygonize

In [ ]:
contours = skimage.measure.find_contours(dem_thresh_filled)
print(f'{len(contours)} contours found')


In [ ]:
# Convert contour coordinates from image to map, polygonize contours 
def image_to_map_coordinates(image, x, y):
    # Extract the affine transform from the dataset attributes
    transform = image.rio.transform()
    # Create the transform if it is missing from the image
    if not transform:
        import affine
        x_res = image.x.data[1] - dem.x.data[0]
        y_res = image.y.data[1] - dem.y.data[0]
        x_min = image.x.data.min()
        y_max = image.y.data.max()  # y_max because y coordinates are typically decreasing
        transform = affine.Affine.translation(x_min, y_max) * affine.Affine.scale(x_res, -y_res)
    # Apply the transform to the x and y coordinates
    map_x, map_y = transform * (x, y)
    return map_x, map_y
# Iterate over contours
polys_list = []
for contour in contours:
    contour_x, contour_y = contour[:,0], contour[:,1]
    contour_map_x, contour_map_y = image_to_map_coordinates(dem_ds, contour_x, contour_y)
    polys_list.append(Polygon(list(zip(contour_map_x, contour_map_y))))

# Filter polygons by area threshold
area_thresh = 1e3
polys_list_filt = [poly for poly in polys_list if poly.area > area_thresh]
poly_areas = np.array([poly.area for poly in polys_list_filt])
print(f'{len(polys_list_filt)} remain after filtering by area')

# Convert polygons to geopandas.GeoDataFrame
polys_gdf = gpd.GeoDataFrame(geometry=[MultiPolygon(polys_list_filt)], crs=f'EPSG:{dem_ds.rio.crs.to_epsg()}')

fig, ax = plt.subplots(1, 2, figsize=(12,6))
polys_gdf.plot(ax=ax[0])
ax[1].hist(poly_areas, bins=20)
ax[1].set_xlabel('Polygon area [m$^2$]')
ax[1].set_ylabel('Count')
plt.show()

## Save to file

In [ ]:
out_fn = dem_fn.replace('.tif', '_roads.shp')
polys_gdf.to_file(out_fn, index=False)
print('Roads shapefile saved to file:', out_fn)